In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import pickle

In [ ]:
%reload_ext autoreload
%autoreload 2

import sys
 
# setting path
sys.path.append('../shared')

from progress import ProgressEstimator
from progsnap import ProgSnap2Dataset
from progsnap import PS2
from progsnap import EventType
from database import CSVDataProvider

In [ ]:
submit_columns = [EventType.Submit, EventType.RunProgram, 'Project.Submit']
problem_ids = [3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,  14,  15,
             16,  17,  18,  19,  20,  21,  22,  23,  24,  30,  31,  33,  35,
             36,  37,  39,  41,  42,  45,  47,  48,  49,  50,  52,  54,  55,
             56,  57,  58,  59,  60,  61,  62,  63,  64,  67,  68,  69,  73,
             75,  78,  79,  80,  81,  82,  83,  84,  87,  88,  89,  90,  91,
             92, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 118, 119,
            120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132,
            133, 134, 135, 136, 137, 138, 139, 140]
problem_ids = problem_ids[:10]
problem_id_column = PS2.ProblemID
code_column = PS2.Code
data_folder = "data/pcrs-f18/"
data_out_folder = "../server/data/PCRS/"

In [ ]:
dataset = ProgSnap2Dataset(CSVDataProvider(data_folder))

In [ ]:
from preprocess import SimpleAIFBuilder

def build_problem(problem_id):

    builder = SimpleAIFBuilder(
        problem_id,
        code_column=code_column, 
        problem_id_column=problem_id_column
    )
    builder.build(dataset)

    with open(f'{data_out_folder}model-{problem_id}.pkl', 'wb') as f:
        pickle.dump(builder.get_trained_classifier(), f)

    estimator = builder.get_trained_progress_model()
    with open(f'{data_out_folder}/progress-{problem_id}.pkl', 'wb') as f:
        pickle.dump(estimator, f)

In [ ]:
for problem_id in problem_ids:
    print(f"Building problem {problem_id}")
    build_problem(problem_id)
    print(f"Built problem {problem_id}")